# Generative Adversarial Networks 

## Training a GAN on the Generative Dog Images dataset from the corresponding Kaggle Competition.

### Note: Can receive a OOM error when training the GAn if there is insufficient VRAM available.

### Dataset contains images and their respective bounding boxes in annotation files.

### Train the discriminator to reduce the error when trying to recognise the difference between generated images (Some of which are Fake) and the real images as much as possible. To train the discriminator, noise is introduced into the generator
### in the form of fake images meant to fool the discriminator network. The approach is game theoretic which means that if either the discriminator or the generator improves during the training session, (that is, the generator becomes more adept
### at fooling the discriminator by introducing more complex noise or the discriminator becomes more adept at recognising the difference between real and fake images) the other has to up the ante in order to be competent.

In [ ]:
import numpy as np
import pandas as pd 
import keras
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Concatenate
from keras.backend import random_normal,ones_like,zeros_like,mean
from keras.backend import get_session
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import concatenate
from keras.initializers import TruncatedNormal
from keras.callbacks import LearningRateScheduler, EarlyStopping, History
from PIL import Image
import warnings
import os
import time
from glob import glob
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd, os
import xml.etree.ElementTree as ET 
import matplotlib.pyplot as plt, zipfile 
from PIL import Image 
from glob import glob

## Reading the Data and Annotations

In [ ]:
IMS = os.listdir('Data/all-dogs/')
breeds = os.listdir('Data/Annotation/')

idxArray = 0; namesArray = []
imagesArray = np.zeros((25000,64,64,3))

print(breeds)

## Read the corresponding bounding boxes for each breed

In [ ]:
DOGS = False
if DOGS:
    for breed in breeds:
        for dog in os.listdir('Data/Annotation/'+ breed):
            try: img = Image.open('all-dogs/'+ dog + '.jpg')
            except: continue

            tree = ET.parse('Data/Annotation/'+ breed + '/'+ dog)
            root = tree.getroot()
            objects = root.findall('object')

            for o in objects:
                boundbox = o.find('bndbox')
                xmin = int(boundbox.find('xmin').text)
                xmax = int(boundbox.find('xmax').text)
                ymin = int(boundbox.find('ymin').text)
                ymax = int(boundbox.find('ymax').text)

                w = np.min((xmax-xmin, ymax-ymin))
                im2 = img.crop((xmin, ymin, xmin+w, ymin+w))
                im2 = im2.resize((64,64), Image.ANTIALIAS)

                imagesArray[idxArray,:,:,:] = np.asarray(im2)

                namesArray.append(breed)
                idxArray+=1

else:
    IMAGES = np.sort(IMS)
    np.random.seed(810)
    x = np.random.choice(np.arange(20579),10000)
    np.random.seed(None)
    for k in range(len(x)):
        img = Image.open('Data/all-dogs/' + IMAGES[x[k]])
        w = img.size[0]; h = img.size[1]
        if (k%2==0)|(k%3==0):
            w2 = 100; h2 = int(h/(w/100))
            a = 18; b = 0          
        else:
            a=0; b=0
            if w<h:
                w2 = 64; h2 = int((64/w)*h)
                b = (h2-64)//2
            else:
                h2 = 64; w2 = int((64/h)*w)
                a = (w2-64)//2
        img = img.resize((w2,h2), Image.ANTIALIAS)
        img = img.crop((0+a, 0+b, 64+a, 64+b))    
        imagesArray[idxArray,:,:,:] = np.asarray(img)
        namesArray.append(IMAGES[x[k]])
        idxArray += 1
x = np.random.randint(0, idxArray, 25)
for k in range(5):
    plt.figure(figsize=(15,3))
    for j in range(5):
        plt.subplot(1, 5, j+1)
        img = Image.fromarray(imagesArray[x[k*5+j], :, :, :].astype('uint8'))
        plt.axis('off')
        plt.imshow(img)
    plt.show()





In [ ]:
IMG_SIZE = Input((12288,))
IMG_SIZE_2 = Input((10000,))
NOISE = 10000

BATCH = 64

# Build Discriminator

In [ ]:
def discriminatorFn():
    input_layer=Dense(12288, activation='sigmoid')(IMG_SIZE_2)
    input_layer=Reshape((2,12288,1))(concatenate([IMG_SIZE, input_layer]))
    discriminator=Conv2D(filters=1, kernel_size=[2,1], use_bias=False, name = 'layer_1')(input_layer)
    out = Flatten()(discriminator)
    return out

In [ ]:
model = discriminatorFn()
model_dis = Model([IMG_SIZE,IMG_SIZE_2], model)
model_dis.get_layer('layer_1').trainable = False
model_dis.get_layer('layer_1').set_weights([np.array([[[[-1.0 ]]],[[[1.0]]]])])
model_dis.summary()
model_dis.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
y_train = (imagesArray[:10000, :, :, :]/255.).reshape((-1,12288))
x_train = np.zeros((10000,10000))
for i in range(10000): 
    x_train[i,1] = 1
zeros = np.zeros((10000, 12288))

lrate = 0.5

for k in range(5):
    LR_S = LearningRateScheduler(lambda x: lrate)
    h = model_dis.fit([zeros, x_train], y_train, epochs = 100, batch_size = BATCH, callbacks = [LR_S], verbose = 0)
    if h.history['loss'][-1] < 0.533: lrate = 0.1

## Sample predictions from the discriminator after training.

In [ ]:
for k in range(5):
    for j in range(5):
        base = np.zeros((10000))
        base[np.random.randint(10000)] = 1
        img = model_dis.predict([zeros[0,:].reshape((-1,12288)), base.reshape((-1,10000))]).reshape((-1,64,64,3))
        img = Image.fromarray( (255*img).astype('uint8').reshape((64,64,3)))

## Build the Generator

In [ ]:
def model_gen(noise_shape=(NOISE,)):
    in_layer = Input(noise_shape)
    gen = Dense(12288, activation='linear')(in_layer)

    model = Model(inputs=in_layer, outputs = [gen, Reshape((10000, ))(in_layer)])
    model.summary()
    return model

    

In [ ]:
model_generator = model_gen(noise_shape=(NOISE,))

## Build the GAN

In [ ]:
model_dis.trainable = False
ins = Input(shape = (NOISE,))
img = model_generator(ins)
real = model_dis(img)

gan = Model(ins, real)
gan.get_layer('model').get_layer('layer_1').set_weights([np.array([[[[-1]]], [[[255,]]]])])
gan.compile(optimizer=Adam(5), loss = 'mean_squared_error')


gan.summary()

In [ ]:
train = np.zeros((10000,10000))
for i in range(10000): train[i,i]=1
zeros = np.zeros((10000, 12288))

steps = 50


## Start training the GAN

In [ ]:
warnings.filterwarnings("ignore")

lr = 5.

for step in range(steps):

    LR_S = LearningRateScheduler(lambda x: lr)
    h = gan.fit(train, zeros, epochs = 1, batch_size=256, callbacks=[LR_S], verbose=0)

    if (step<10)|(step%5==4):
        print ("Step: {}/{} [G loss: {:.4f}]".format(
                     (step+1)*10, steps*10, h.history['loss'][-1]))
    
    if h.history['loss'][-1] < 25: lr = 1. 
    if h.history['loss'][-1] < 1.5: lr = 0.01

    if step<10:
        plt.figure(figsize=(15,3))
        for j in range(5):
            zz = np.zeros((10000))
            zz[np.random.randint(10000)] = 1
            plt.subplot(1,5,j+1)
            img = model_generator.predict(zz.reshape((-1,10000)))[0].reshape((-1,64,64,3))
            img = Image.fromarray( (img).astype('uint8').reshape((64,64,3)))
            plt.axis('off')
            plt.imshow(img)
        plt.show()    
